In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw, drop_invalid=True)
dev_pack_processed = preprocessor.transform(dev_pack_raw, drop_invalid=False)
test_pack_processed = preprocessor.transform(test_pack_raw, drop_invalid=False)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3655.05it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2409.04it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 1753083.17it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3654.57it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2397.24it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 36506.61it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3862.54it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f04166207f0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f042454fa20>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675,
 'input_shapes': [(10,), (40,)]}

In [5]:
model = mz.contrib.models.MatchLSTM()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['fc_num_units'] = 100
model.params['lstm_num_units'] = 100
model.params['dropout_rate'] = 0.5
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
print(model.params)

model_class                   <class 'matchzoo.contrib.models.match_lstm.MatchLSTM'>
input_shapes                  [(10,), (40,)]
task                          Ranking Task
optimizer                     adadelta
with_embedding                True
embedding_input_dim           16675
embedding_output_dim          300
embedding_trainable           True
lstm_num_units                100
fc_num_units                  100
dropout_rate                  0.5


In [6]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             5002500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
lstm_left (LSTM)                (None, 10, 100)      160400      embedding[0][0]                  
__________

In [7]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [8]:
test_x, test_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_x))

In [9]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 102


In [10]:
history = model.fit_generator(train_generator, epochs=10, callbacks=[evaluate], workers=4, use_multiprocessing=True)

Epoch 1/10
102/102 [==============================] - 34s 333ms/step - loss: 0.9173
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4203580039672582 - normalized_discounted_cumulative_gain@5(0.0): 0.496241570575916 - mean_average_precision(0.0): 0.4580415855965
Epoch 2/10
102/102 [==============================] - 27s 263ms/step - loss: 0.6855
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5897753589225543 - normalized_discounted_cumulative_gain@5(0.0): 0.6490551789962096 - mean_average_precision(0.0): 0.6101337166793529
Epoch 3/10
102/102 [==============================] - 26s 255ms/step - loss: 0.6049
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6052789016600058 - normalized_discounted_cumulative_gain@5(0.0): 0.6571307409264713 - mean_average_precision(0.0): 0.6111243714091814
Epoch 4/10
102/102 [==============================] - 27s 269ms/step - loss: 0.4786
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5944665170448176 - normali